#NO - EA Inflation Core

In [1]:
# === Imports ===
import pandas as pd
import numpy as np
import requests, gzip, io, re

# === Eurostat SDMX TSV (komprimert) ===
URL = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/prc_hicp_midx?format=TSV&compressed=true"

# === Last ned og pakk ut ===
r = requests.get(URL, timeout=120)
r.raise_for_status()
with gzip.open(io.BytesIO(r.content), 'rt') as f:
    df_raw = pd.read_csv(f, sep='\t')

# --- Del opp dimensjonskolonnen: 'freq,unit,coicop,geo\\TIME_PERIOD' ---
dimcol = df_raw.columns[0]
df_raw[['freq','unit','coicop','geo']] = df_raw[dimcol].str.split(',', expand=True)
df_raw = df_raw.drop(columns=[dimcol])

# --- Finn tidskolonner (YYYY-MM med ev. trailing space) og smelt til long ---
time_cols = [c for c in df_raw.columns if re.match(r'^\d{4}-\d{2}\s*$', c)]
df_long = df_raw.melt(
    id_vars=['freq','unit','coicop','geo'],
    value_vars=time_cols,
    var_name='time',
    value_name='value'
)

# --- Rens verdier: fjern tom/':' og trekk ut kun tall (kan ha flagg som 'p') ---
df_long['value'] = df_long['value'].astype(str).str.strip()
df_long = df_long[(df_long['value'] != ':') & (df_long['value'] != '')]
num = df_long['value'].str.extract(r'^\s*([-]?\d+(?:\.\d+)?)')
df_long = df_long[~num[0].isna()].copy()
df_long['value'] = num[0].astype(float)

# --- Standardiser tid ---
df_long['time'] = df_long['time'].str.strip()
df_long['time'] = pd.to_datetime(df_long['time'], format="%Y-%m", errors='coerce')
df_long = df_long.dropna(subset=['time'])

# === Hjelpefunksjoner ===
def best_unit(df, prefer=('I15','I05','I96')):
    units = df['unit'].dropna().unique().tolist()
    for u in prefer:
        if u in units:
            return u
    return units[0] if units else None

def get_series(df, geo, coicop, start_year=1999, end_year=2025):
    sub = df.query("geo == @geo and coicop == @coicop").copy()
    if sub.empty:
        raise ValueError(f"Ingen rader for geo={geo}, coicop={coicop}")
    u = best_unit(sub)
    sub = sub[sub['unit'] == u].copy()
    sub = sub[(sub['time'].dt.year >= start_year) & (sub['time'].dt.year <= end_year)]
    sub = sub.sort_values('time').reset_index(drop=True)
    return sub[['time','value']].assign(unit=u)

# === Serier ===
# p_t  : Norge kjerne-HICP (eks. energi og mat) → TOT_X_NRG_FOOD
no_core = get_series(df_long, geo='NO', coicop='TOT_X_NRG_FOOD', start_year=1999, end_year=2025)
print(f"Norge kjerne-HICP (TOT_X_NRG_FOOD) unit: {no_core['unit'].iloc[0]}")

# p_t* : Euro-området HICP total → CP00
ea_all = get_series(df_long, geo='EA', coicop='CP00', start_year=1999, end_year=2025)
print(f"Euro-området HICP total (CP00) unit: {ea_all['unit'].iloc[0]}")

# === Bygg full månedlig rekke og ffill hull ===
# NB: NO core starter i 1999-12; derfor starter vi indeksen i 1999-12 for å unngå uønsket bfill.
full_index = pd.date_range(start="1999-12-01", end="2025-12-01", freq="MS")

df = full_index.to_frame(name="time")
df = df.merge(no_core[['time','value']].rename(columns={'value':'hicp_no_core'}), on='time', how='left')
df = df.merge(ea_all[['time','value']].rename(columns={'value':'hicp_ea_all'}), on='time', how='left')

# Fyll eventuelle mellomliggende mangler for hver serie
df[['hicp_no_core','hicp_ea_all']] = df[['hicp_no_core','hicp_ea_all']].ffill()

# === Log-kolonner med tydelige navn ===
df['p_no_core_log'] = np.log(df['hicp_no_core'])   # p_t
df['p_ea_all_log']  = np.log(df['hicp_ea_all'])    # p_t*

# === Velg både nivå og log, i tydelig rekkefølge ===
out = df[['time', 'hicp_no_core', 'hicp_ea_all', 'p_no_core_log', 'p_ea_all_log']].copy()

# === Sjekk og lagre ===
print(out.head(5))
print(out.tail(5))
print(f"Antall måneder: {len(out)}  | Periode: {out['time'].min().date()} → {out['time'].max().date()}")

#out.to_csv("RER_inputs_Eurostat_1999-12_to_2025-12_ffill.csv", index=False)
#print("Lagret: RER_inputs_Eurostat_1999-12_to_2025-12_ffill.csv")


Norge kjerne-HICP (TOT_X_NRG_FOOD) unit: I15
Euro-området HICP total (CP00) unit: I15
        time  hicp_no_core  hicp_ea_all  p_no_core_log  p_ea_all_log
0 1999-12-01          79.9        75.09       4.380776      4.318687
1 2000-01-01          79.7        75.13       4.378270      4.319220
2 2000-02-01          80.1        75.37       4.383276      4.322409
3 2000-03-01          80.3        75.60       4.385770      4.325456
4 2000-04-01          80.8        75.67       4.391977      4.326382
          time  hicp_no_core  hicp_ea_all  p_no_core_log  p_ea_all_log
308 2025-08-01         133.6       129.31       4.894850      4.862213
309 2025-09-01         134.1       129.43       4.898586      4.863140
310 2025-10-01         134.1       129.43       4.898586      4.863140
311 2025-11-01         134.1       129.43       4.898586      4.863140
312 2025-12-01         134.1       129.43       4.898586      4.863140
Antall måneder: 313  | Periode: 1999-12-01 → 2025-12-01


#EU NOK

In [2]:
import pandas as pd

# === 1) Last ned data fra Norges Bank (2000–2025) ===
url = ("https://data.norges-bank.no/api/data/EXR/B.EUR.NOK.SP"
       "?format=csv&bom=include&apisrc=nbi"
       "&startPeriod=2000-01-01&endPeriod=2025-12-31&locale=no")

# Les CSV (semikolon-separator, desimalkomma)
df = pd.read_csv(url, sep=';', encoding='utf-8-sig', decimal=',')

# === 2) Plukk ut dato og kurs ===
df = (df[['TIME_PERIOD', 'OBS_VALUE']]
        .rename(columns={'TIME_PERIOD': 'DATE', 'OBS_VALUE': 'EUR_NOK'}))

df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
df = df.dropna(subset=['DATE','EUR_NOK']).sort_values('DATE').set_index('DATE')

# === 3) Lag full daglig index og fyll hull ===
full_idx = pd.date_range(df.index.min(), df.index.max(), freq='D')
df_daily = df.reindex(full_idx)

df_daily['EUR_NOK'] = df_daily['EUR_NOK'].ffill()
df_daily.index.name = 'DATE'
df_daily = df_daily.asfreq('D')

# === 4) Sjekk resultat ===
print(df_daily.head(10))
print(df_daily.tail(10))
print(f"\nAntall dager: {len(df_daily)}  | Periode: {df_daily.index.min().date()} → {df_daily.index.max().date()}")

# (Valgfritt) lagre for videre bruk
# df_daily.to_csv('eur_nok_daily_ffill.csv')
# df_daily.to_parquet('eur_nok_daily_ffill.parquet')


            EUR_NOK
DATE               
2000-01-03   8.0620
2000-01-04   8.1500
2000-01-05   8.2060
2000-01-06   8.2030
2000-01-07   8.1945
2000-01-08   8.1945
2000-01-09   8.1945
2000-01-10   8.1900
2000-01-11   8.2075
2000-01-12   8.2160
            EUR_NOK
DATE               
2025-10-18  11.7293
2025-10-19  11.7293
2025-10-20  11.7415
2025-10-21  11.6693
2025-10-22  11.6430
2025-10-23  11.5830
2025-10-24  11.6185
2025-10-25  11.6185
2025-10-26  11.6185
2025-10-27  11.6320

Antall dager: 9430  | Periode: 2000-01-03 → 2025-10-27


#Merge EU/NOK price with Inflation

In [3]:
# === 1) Gjør HICP (out) daglig med ffill ===
# Start og slutt defineres fra out
daily_idx = pd.date_range(start=out['time'].min(), end=out['time'].max(), freq='D')

out_daily = (
    out.set_index('time')
       .reindex(daily_idx)     # legg til alle dager
       .ffill()                # behold siste månedsverdi frem til ny måned
)

out_daily.index.name = 'DATE'

# === 2) Merge med Norges Bank daglige data (df_daily) ===
merged = (
    df_daily[['EUR_NOK']]
    .merge(out_daily, left_index=True, right_index=True, how='left')
)

# === 3) Legg til log av valutakurs ===
merged['s_eurnok_log'] = np.log(merged['EUR_NOK'])

# === 4) Sjekk resultat ===
print(merged.head(10))
print(merged.tail(10))
print(f"\nAntall dager: {len(merged)} | Periode: {merged.index.min().date()} → {merged.index.max().date()}")

# (Valgfritt) lagre for analyse videre
# merged.to_csv("merged_eurnok_hicp_daily.csv")


            EUR_NOK  hicp_no_core  hicp_ea_all  p_no_core_log  p_ea_all_log  \
DATE                                                                          
2000-01-03   8.0620          79.7        75.13        4.37827       4.31922   
2000-01-04   8.1500          79.7        75.13        4.37827       4.31922   
2000-01-05   8.2060          79.7        75.13        4.37827       4.31922   
2000-01-06   8.2030          79.7        75.13        4.37827       4.31922   
2000-01-07   8.1945          79.7        75.13        4.37827       4.31922   
2000-01-08   8.1945          79.7        75.13        4.37827       4.31922   
2000-01-09   8.1945          79.7        75.13        4.37827       4.31922   
2000-01-10   8.1900          79.7        75.13        4.37827       4.31922   
2000-01-11   8.2075          79.7        75.13        4.37827       4.31922   
2000-01-12   8.2160          79.7        75.13        4.37827       4.31922   

            s_eurnok_log  
DATE                    

#Reell valutakurs (q) & Inflasjonsdifferanse (dπ)

In [4]:
import pandas as pd
import numpy as np

# === 1) Gjør HICP (out) daglig med ffill (nivåer) ===
# Start og slutt defineres fra 'out'
daily_idx = pd.date_range(start=out['time'].min(), end=out['time'].max(), freq='D')
out_daily = (
    out.set_index('time')
       .reindex(daily_idx)   # legg til alle dager
       .ffill()              # behold siste månedsnivå frem til ny måned
)
out_daily.index.name = 'DATE'

# === 2) Merge med Norges Bank daglige data (df_daily) ===
merged = (
    df_daily[['EUR_NOK']]
    .merge(out_daily, left_index=True, right_index=True, how='left')
)

# === 3) Daglige log-nivåer og reell valutakurs ===
# s_t: log NOK/EUR
merged['st'] = np.log(merged['EUR_NOK'])

# p_t (Norge) og p_t* (Euro): log av RÅ indekser (unngår avrundingsfeil)
merged['pt']      = np.log(merged['hicp_no_core'])
merged['pt_star'] = np.log(merged['hicp_ea_all'])

# q_t = s_t - (p_t - p_t*)
merged['qt'] = merged['st'] - (merged['pt'] - merged['pt_star'])

# === 4) Måned-til-måned inflasjon (Δlog) på månedsslutt, mappet til daglig i SAMME måned ===
# Nivåer på månedsslutt
no_me = merged['hicp_no_core'].resample('ME').last()
ea_me = merged['hicp_ea_all' ].resample('ME').last()

# Log-nivåer (månedsslutt) og Δlog (måned-til-måned)
p_no_me = np.log(no_me)
p_ea_me = np.log(ea_me)
pi_no_m = p_no_me.diff(1)          # π_t  (Norge)
pi_ea_m = p_ea_me.diff(1)          # π_t* (Euro)
dpi_m   = pi_no_m - pi_ea_m        # dπ_t

# Map til daglig slik at verdien gjelder for HELE SAMME MÅNED (bfill fra månedsslutt bakover)
didx   = merged.index
end_me = didx.max() + pd.offsets.MonthEnd(0)                  # utvid til siste månedsslutt som dekkes av daglig data
drng   = pd.date_range(pi_no_m.index.min(), end_me, freq='D') # midlertidig daglig akse for mapping

def to_daily_same_month(s, daily_index):
    # bfill fyller månedsslutt-verdien bakover i samme måned
    daily_full = s.reindex(drng).bfill()
    return daily_full.reindex(daily_index)

merged['pi_t']      = to_daily_same_month(pi_no_m, didx)      # inflasjon Norge (Δlog KPI-JAE)
merged['pi_t_star'] = to_daily_same_month(pi_ea_m, didx)      # inflasjon Euro  (Δlog HICP)
merged['dpi_t']     = to_daily_same_month(dpi_m,   didx)      # inflasjonsdifferanse = π_t - π_t*

# === 5) Rydding for modellbruk: fjern NaN-rader i nødvendige kolonner ===
# Første måned med Δlog mangler referanse (første differanse) → kan bli NaN: droppes her.
needed_cols = [
    'EUR_NOK', 'hicp_no_core', 'hicp_ea_all',  # nivåer
    'st', 'pt', 'pt_star', 'qt',               # log-nivåer / reell valutakurs
    'pi_t', 'pi_t_star', 'dpi_t'               # inflasjon og differanse
]
final_df = merged.dropna(subset=needed_cols).copy()

# === 6) (Valgfritt) Sjekk/visning ===
print(final_df[needed_cols].head(12))
print(final_df[needed_cols].tail(12))
print(f"\nAntall dager (opprinnelig): {len(merged)} | Periode: {merged.index.min().date()} → {merged.index.max().date()}")
print(f"Antall dager (etter NaN-fjerning): {len(final_df)}")

# === 7) Lagre datasett klart til modell ===
# Bruk final_df (uten NaN) i modellen.
# final_df.to_csv('merged_q_inflasjon_ready.csv')


            EUR_NOK  hicp_no_core  hicp_ea_all        st        pt   pt_star  \
DATE                                                                           
2000-01-31   8.0825          79.7        75.13  2.089701  4.378270  4.319220   
2000-02-01   8.0730          80.1        75.37  2.088525  4.383276  4.322409   
2000-02-02   8.0175          80.1        75.37  2.081627  4.383276  4.322409   
2000-02-03   8.0475          80.1        75.37  2.085361  4.383276  4.322409   
2000-02-04   8.0830          80.1        75.37  2.089763  4.383276  4.322409   
2000-02-05   8.0830          80.1        75.37  2.089763  4.383276  4.322409   
2000-02-06   8.0830          80.1        75.37  2.089763  4.383276  4.322409   
2000-02-07   8.0590          80.1        75.37  2.086789  4.383276  4.322409   
2000-02-08   8.0720          80.1        75.37  2.088401  4.383276  4.322409   
2000-02-09   8.0825          80.1        75.37  2.089701  4.383276  4.322409   
2000-02-10   8.0695          80.1       

In [5]:
# === Kun EUR_NOK, Q og d_pi ===
import numpy as np
import pandas as pd

# 1) Q = reell valutakurs = s_t - (p_t - p_t*)
merged['st']      = np.log(merged['EUR_NOK'])          # s_t
merged['pt']      = np.log(merged['hicp_no_core'])     # p_t
merged['pt_star'] = np.log(merged['hicp_ea_all'])      # p_t*
merged['Q']       = merged['st'] - (merged['pt'] - merged['pt_star'])

# 2) d_pi = inflasjonsdifferanse = π_t - π_t*
#    (Δlog av månedsslutt-nivåer, mappet til daglig i samme måned)
no_me = merged['hicp_no_core'].resample('ME').last()
ea_me = merged['hicp_ea_all' ].resample('ME').last()

pi_no_m = np.log(no_me).diff(1)
pi_ea_m = np.log(ea_me).diff(1)
d_pi_m  = pi_no_m - pi_ea_m

# Map til daglig: bfill fra månedsslutt bakover slik at verdien gjelder for hele samme måned
didx   = merged.index
end_me = didx.max() + pd.offsets.MonthEnd(0)
drng   = pd.date_range(d_pi_m.index.min(), end_me, freq='D')

d_pi_daily = d_pi_m.reindex(drng).bfill().reindex(didx)
merged['d_pi'] = d_pi_daily

# 3) Endelig datasett til modell
final_small = merged[['EUR_NOK', 'Q', 'd_pi']].dropna().copy()

# (Valgfritt) sjekk/vis
print(final_small.head(12))
print(final_small.tail(12))
print(f"\nRader totalt: {len(merged)}  | Rader i final_small (uten NaN): {len(final_small)}")

# (Valgfritt) lagre
# final_small.to_csv('eurnok_Q_dpi_ready.csv', index=True)


            EUR_NOK         Q      d_pi
DATE                                   
2000-01-31   8.0825  2.030652  0.001817
2000-02-01   8.0730  2.027659  0.001817
2000-02-02   8.0175  2.020760  0.001817
2000-02-03   8.0475  2.024495  0.001817
2000-02-04   8.0830  2.028897  0.001817
2000-02-05   8.0830  2.028897  0.001817
2000-02-06   8.0830  2.028897  0.001817
2000-02-07   8.0590  2.025923  0.001817
2000-02-08   8.0720  2.027535  0.001817
2000-02-09   8.0825  2.028835  0.001817
2000-02-10   8.0695  2.027225  0.001817
2000-02-11   8.0395  2.023500  0.001817
            EUR_NOK         Q  d_pi
DATE                               
2025-10-16  11.7340  2.427045   0.0
2025-10-17  11.7293  2.426644   0.0
2025-10-18  11.7293  2.426644   0.0
2025-10-19  11.7293  2.426644   0.0
2025-10-20  11.7415  2.427684   0.0
2025-10-21  11.6693  2.421516   0.0
2025-10-22  11.6430  2.419260   0.0
2025-10-23  11.5830  2.414093   0.0
2025-10-24  11.6185  2.417153   0.0
2025-10-25  11.6185  2.417153   0.0
2025-10-

#Styringsrenter


In [6]:
import pandas as pd

# === 1) Hent styringsrente (Norges Bank API) ===
url = (
    "https://data.norges-bank.no/api/data/IR/B.KPRA.SD.R"
    "?apisrc=qb&format=csv&startPeriod=1996-01-01&endPeriod=2025-09-26&locale=no&bom=include"
)

# NB-CSV er semikolonskilt og kan ha BOM; bruk riktig sep/encoding
df_rate = pd.read_csv(url, sep=";", encoding="utf-8-sig", engine="python")

# Normaliser kolonnenavn (trim)
df_rate.columns = [c.strip() for c in df_rate.columns]

# Finn tids- og verdikolonne robust (NO/EN-varianter)
time_candidates  = ["TIME_PERIOD", "Tid", "TIME", "Date", "PERIOD"]
value_candidates = ["OBS_VALUE", "Observasjonsverdi", "Value", "VALUE"]

time_col  = next((c for c in time_candidates  if c in df_rate.columns), None)
value_col = next((c for c in value_candidates if c in df_rate.columns), None)
if time_col is None or value_col is None:
    raise KeyError(f"Finner ikke tids-/verdikolonne i CSV. Kolonner: {df_rate.columns.tolist()}")

# Velg og rens relevante kolonner
rate = df_rate[[time_col, value_col]].copy()
rate.rename(columns={time_col: "DATE", value_col: "policy_rate"}, inplace=True)

# Konverter policy_rate til numerisk (håndter evt. komma-desimaler)
rate["policy_rate"] = pd.to_numeric(rate["policy_rate"], errors="coerce")
if rate["policy_rate"].isna().all():
    rate["policy_rate"] = (
        rate["policy_rate"].astype(str).str.replace(",", ".", regex=False)
    )
    rate["policy_rate"] = pd.to_numeric(rate["policy_rate"], errors="coerce")

# Datoindeks og sortering
rate["DATE"] = pd.to_datetime(rate["DATE"], errors="coerce", utc=False)
rate = rate.dropna(subset=["DATE", "policy_rate"]).sort_values("DATE").set_index("DATE")

# === 2) Gjør tidsserien komplett daglig (inkludér helger/helligdager) ===
# Bruk hele datoperioden som finnes i din eksisterende 'merged' (for å matche modellens kalender).
target_start = merged.index.min()
target_end   = merged.index.max()
target_idx   = pd.date_range(start=target_start, end=target_end, freq="D")

policy_rate_daily = (
    rate.reindex(target_idx)  # legg til alle kalenderdager
        .ffill()              # behold siste kjente rente frem til ny observasjon
        .bfill()              # fyll starten dersom merged starter før første observasjon
)
policy_rate_daily.index.name = "DATE"

# === 3) Merge inn i datasettet ditt ===
merged = merged.merge(policy_rate_daily[["policy_rate"]], left_index=True, right_index=True, how="left")

# === 4) (Valgfritt) Sjekk ===
print(merged[["policy_rate"]].head(10))
print(merged[["policy_rate"]].tail(10))
print(f"\npolicy_rate: dekker {merged['policy_rate'].index.min().date()} → {merged['policy_rate'].index.max().date()} | "
      f"NaN: {merged['policy_rate'].isna().sum()}")


            policy_rate
DATE                   
2000-01-03          7.0
2000-01-04          7.0
2000-01-05          7.0
2000-01-06          7.0
2000-01-07          7.0
2000-01-08          7.0
2000-01-09          7.0
2000-01-10          7.0
2000-01-11          7.0
2000-01-12          7.0
            policy_rate
DATE                   
2025-10-18          4.0
2025-10-19          4.0
2025-10-20          4.0
2025-10-21          4.0
2025-10-22          4.0
2025-10-23          4.0
2025-10-24          4.0
2025-10-25          4.0
2025-10-26          4.0
2025-10-27          4.0

policy_rate: dekker 2000-01-03 → 2025-10-27 | NaN: 0


In [7]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# -------------------------------------------------------
# 1) Hent HTML og finn tabellen "Key ECB interest rates"
# -------------------------------------------------------
URL = "https://www.ecb.europa.eu/stats/policy_and_exchange_rates/key_ecb_interest_rates/html/index.en.html"
html = requests.get(URL, timeout=30).text
soup = BeautifulSoup(html, "lxml")

# Finn første tabell som inneholder overskriften "Date (with effect from)"
target_table = None
for tbl in soup.find_all("table"):
    if tbl.find(string=re.compile(r"Date \(with effect from\)", re.I)):
        target_table = tbl
        break
if target_table is None:
    raise RuntimeError("Fant ikke tabellen med 'Date (with effect from)' på ECB-siden.")

# -------------------------------------------------------
# 2) Ekstrahér rader robust (år + dag.mnd. + 3 renter)
#    Håndter fotnoter, bindestreker, minus-tegn, etc.
# -------------------------------------------------------
def clean_text(x: str) -> str:
    if x is None:
        return ""
    x = re.sub(r"\s+", " ", x.strip())
    x = x.replace("−", "-")  # minus-tegn
    x = re.sub(r"\^\{\d+\}", "", x)  # fotnote-superscripts i tekstduken
    return x

def to_float_or_nan(s: str):
    s = s.strip()
    if s in ("-", ""):
        return np.nan
    s = s.replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

# Månedsforkortelser på siden (engelsk med punktum)
MONTH_MAP = {
    "Jan.": "Jan", "Feb.": "Feb", "Mar.": "Mar", "Apr.": "Apr", "May": "May",
    "Jun.": "Jun", "Jul.": "Jul", "Aug.": "Aug", "Sep.": "Sep", "Oct.": "Oct",
    "Nov.": "Nov", "Dec.": "Dec"
}

rows_out = []
current_year = None

for tr in target_table.find_all("tr"):
    tds = [clean_text(td.get_text(" ", strip=True)) for td in tr.find_all(["td", "th"])]
    if not tds:
        continue

    # Hopp over header-linjer med "Fixed rate tenders" etc.
    joined = " ".join(tds)
    if re.search(r"Fixed rate tenders|Variable rate tenders|Minimum bid rate", joined, re.I):
        continue
    if re.search(r"Date \(with effect from\)", joined, re.I):
        continue

    # Vanligvis er formatet:
    # [YEAR, "11 Jun.", Deposit, MRO, Marginal]
    # Noen ganger kan YEAR mangle (blank), da brukes current_year
    # Noen linjer kan slå sammen YEAR + dato i én celle -> håndter via regex.
    year, dstr, dep, mro, mlf = None, None, None, None, None

    # 5 kolonner (normal tilstand): Year, DayMonth, Deposit, MRO, MLF
    if len(tds) >= 5 and re.fullmatch(r"\d{4}", tds[0]):
        year = tds[0]
        dstr = tds[1]
        dep  = tds[2]
        mro  = tds[3]
        mlf  = tds[4]
    else:
        # Prøv å trekke ut "YYYY <day> <Mon.>" fra første/andre celle
        # Søk i alle celler for et mønster "YYYY <d{1,2}> <Mon.>"
        found = None
        for i, cell in enumerate(tds[:2]):  # sjekk de første cellene for dato/år
            m = re.search(r"(\d{4})\s+(\d{1,2})\s+([A-Za-z]{3}\.?)", cell)
            if m:
                found = (m.group(1), m.group(2), m.group(3), i)
                break
        if found:
            year, day, mon_raw, idx = found
            mon = MONTH_MAP.get(mon_raw if mon_raw.endswith(".") else mon_raw + ".", mon_raw.replace(".", ""))
            dstr = f"{day} {mon}"
            # Resterende kolonner etter datocellen
            rest = tds[idx+1:]
            # fisk renter i rekkefølgen dep, mro, mlf om mulig
            if len(rest) >= 3:
                dep, mro, mlf = rest[0], rest[1], rest[2]
        else:
            # Evt. to-cellers rader der første celle bare er år eller blank
            if len(tds) >= 4 and (re.fullmatch(r"\d{4}", tds[0]) or current_year):
                if re.fullmatch(r"\d{4}", tds[0]):
                    current_year = tds[0]
                    dstr, dep, mro, mlf = tds[1], tds[2], tds[3], (tds[4] if len(tds) > 4 else "-")
                else:
                    year = current_year
                    dstr, dep, mro, mlf = tds[0], tds[1], tds[2], (tds[3] if len(tds) > 3 else "-")

    # Fyll år dersom ikke satt eksplisitt
    if year is None and re.fullmatch(r"\d{4}", tds[0]):
        year = tds[0]
    if year is None:
        year = current_year
    else:
        current_year = year

    # Må ha år + dstr og minst én rente
    if not (year and dstr):
        continue

    # Normaliser månedstekst
    parts = dstr.split()
    if len(parts) >= 2:
        day = parts[0]
        mon_raw = parts[1]
        mon = MONTH_MAP.get(mon_raw, mon_raw.replace(".", ""))
        d_iso = pd.to_datetime(f"{day} {mon} {year}", format="%d %b %Y", errors="coerce")
    else:
        d_iso = pd.NaT

    dep_v = to_float_or_nan(dep or "")
    mro_v = to_float_or_nan(mro or "")
    mlf_v = to_float_or_nan(mlf or "")

    rows_out.append({
        "effective_date": d_iso,
        "deposit_facility": dep_v,
        "main_refi": mro_v,
        "marginal_lending": mlf_v
    })

df_ecb = pd.DataFrame(rows_out).dropna(subset=["effective_date"]).sort_values("effective_date").reset_index(drop=True)

# -------------------------------------------------------
# 3) Velg policy rate for EU (Deposit facility fra ECB)
#    (ECB skriver at de styrer via denne fra 2024.)
# -------------------------------------------------------
df_ecb["eu_policy_rate"] = df_ecb["deposit_facility"]

# -------------------------------------------------------
# 4) Lag daglig serie med ffill (og bfill i starten ved behov)
#    Hvis du vil matche merged-kalenderen, bruk merged.index.min/max.
# -------------------------------------------------------
# Velg daglig kalender fra første effektive dato til i dag/eller din slutt
daily_start = df_ecb["effective_date"].min()
daily_end   = df_ecb["effective_date"].max()

daily_idx = pd.date_range(start=daily_start, end=daily_end, freq="D")
daily = (
    df_ecb.set_index("effective_date")[["eu_policy_rate"]]
          .reindex(daily_idx)
          .ffill()
          .bfill()
)
daily.index.name = "DATE"

# -------------------------------------------------------
# 5) (Valgfritt) merge inn i ditt datasett "merged"
# -------------------------------------------------------
# merged = merged.merge(daily, left_index=True, right_index=True, how="left")

# -------------------------------------------------------
# 6) Sjekk
# -------------------------------------------------------
print(daily.head(12))
print(daily.tail(12))
print(f"\nEU policy rate daglig: {daily.index.min().date()} → {daily.index.max().date()} | NaN: {daily['eu_policy_rate'].isna().sum()}")


            eu_policy_rate
DATE                      
1999-11-05             2.0
1999-11-06             2.0
1999-11-07             2.0
1999-11-08             2.0
1999-11-09             2.0
1999-11-10             2.0
1999-11-11             2.0
1999-11-12             2.0
1999-11-13             2.0
1999-11-14             2.0
1999-11-15             2.0
1999-11-16             2.0
            eu_policy_rate
DATE                      
2025-05-31            2.25
2025-06-01            2.25
2025-06-02            2.25
2025-06-03            2.25
2025-06-04            2.25
2025-06-05            2.25
2025-06-06            2.25
2025-06-07            2.25
2025-06-08            2.25
2025-06-09            2.25
2025-06-10            2.25
2025-06-11            2.00

EU policy rate daglig: 1999-11-05 → 2025-06-11 | NaN: 0


In [8]:
# === Rentedifferanse: dI_t = i_t - i_t* ===
import pandas as pd

# 1) Mål-kalender = hele perioden i 'merged'
target_start = merged.index.min()
target_end   = merged.index.max()
target_idx   = pd.date_range(start=target_start, end=target_end, freq='D')

# 2) Reindekser begge renter til daglig, ffill og bfill for komplett serie
nor_rate_daily = (
    policy_rate_daily.reindex(target_idx)
                     .ffill()
                     .bfill()
    .rename(columns={'policy_rate': 'i_t'})  # Norge
)
eu_rate_daily = (
    daily.reindex(target_idx)
         .ffill()
         .bfill()
    .rename(columns={'eu_policy_rate': 'i_t_star'})  # Euroområdet (ECB)
)

# 3) Slå sammen og beregn differanse
rates = pd.concat([nor_rate_daily, eu_rate_daily], axis=1)
rates.index.name = 'DATE'
rates['dI_t'] = rates['i_t'] - rates['i_t_star']

# 4) (Valgfritt) slå inn i ditt hoved-datasett og fjern NaN der det trengs
merged = merged.merge(rates, left_index=True, right_index=True, how='left')

# 5) Klargjør liten tabell til modellbruk (kun renteinfo, uten NaN)
rates_small = rates.dropna(subset=['i_t','i_t_star','dI_t']).copy()

# 6) Sjekk
print(rates_small.head(12))
print(rates_small.tail(12))
print(f"\nRenter (komplett daglig): {rates_small.index.min().date()} → {rates_small.index.max().date()} | "
      f"NaN i i_t: {rates['i_t'].isna().sum()}, i_t*: {rates['i_t_star'].isna().sum()}, dI_t: {rates['dI_t'].isna().sum()}")


            i_t  i_t_star  dI_t
DATE                           
2000-01-03  7.0       2.0   5.0
2000-01-04  7.0       2.0   5.0
2000-01-05  7.0       2.0   5.0
2000-01-06  7.0       2.0   5.0
2000-01-07  7.0       2.0   5.0
2000-01-08  7.0       2.0   5.0
2000-01-09  7.0       2.0   5.0
2000-01-10  7.0       2.0   5.0
2000-01-11  7.0       2.0   5.0
2000-01-12  7.0       2.0   5.0
2000-01-13  7.0       2.0   5.0
2000-01-14  7.0       2.0   5.0
            i_t  i_t_star  dI_t
DATE                           
2025-10-16  4.0       2.0   2.0
2025-10-17  4.0       2.0   2.0
2025-10-18  4.0       2.0   2.0
2025-10-19  4.0       2.0   2.0
2025-10-20  4.0       2.0   2.0
2025-10-21  4.0       2.0   2.0
2025-10-22  4.0       2.0   2.0
2025-10-23  4.0       2.0   2.0
2025-10-24  4.0       2.0   2.0
2025-10-25  4.0       2.0   2.0
2025-10-26  4.0       2.0   2.0
2025-10-27  4.0       2.0   2.0

Renter (komplett daglig): 2000-01-03 → 2025-10-27 | NaN i i_t: 0, i_t*: 0, dI_t: 0


#Daglig Data Endogene

In [9]:
# === Slå sammen final_small med renter (left join) ===
final_with_rates = (
    final_small
    .merge(rates[["dI_t"]], left_index=True, right_index=True, how="left")
    .dropna(subset=["dI_t"])  # behold kun rader der rentedifferanse er definert
)

# Velg kun de kolonnene du trenger
final_with_rates = final_with_rates[["EUR_NOK", "Q", "d_pi", "dI_t"]]

# --- Sjekk ---
print(final_with_rates.head(12))
print(final_with_rates.tail(12))
print(f"\nTotalt rader: {len(final_with_rates)} | "
      f"Periode: {final_with_rates.index.min().date()} → {final_with_rates.index.max().date()}")

# (Valgfritt) lagre
# final_with_rates.to_csv("final_dataset_ready.csv", index=True)


            EUR_NOK         Q      d_pi  dI_t
DATE                                         
2000-01-31   8.0825  2.030652  0.001817   5.0
2000-02-01   8.0730  2.027659  0.001817   5.0
2000-02-02   8.0175  2.020760  0.001817   5.0
2000-02-03   8.0475  2.024495  0.001817   5.0
2000-02-04   8.0830  2.028897  0.001817   5.0
2000-02-05   8.0830  2.028897  0.001817   5.0
2000-02-06   8.0830  2.028897  0.001817   5.0
2000-02-07   8.0590  2.025923  0.001817   5.0
2000-02-08   8.0720  2.027535  0.001817   5.0
2000-02-09   8.0825  2.028835  0.001817   5.0
2000-02-10   8.0695  2.027225  0.001817   5.0
2000-02-11   8.0395  2.023500  0.001817   5.0
            EUR_NOK         Q  d_pi  dI_t
DATE                                     
2025-10-16  11.7340  2.427045   0.0   2.0
2025-10-17  11.7293  2.426644   0.0   2.0
2025-10-18  11.7293  2.426644   0.0   2.0
2025-10-19  11.7293  2.426644   0.0   2.0
2025-10-20  11.7415  2.427684   0.0   2.0
2025-10-21  11.6693  2.421516   0.0   2.0
2025-10-22  11.6430 

#Måntlig data

In [10]:
# === Aggreger til månedlig frekvens ===

# Resample til månedsslutt (ME = month end)
final_monthly = pd.DataFrame({
    "EUR_NOK": final_with_rates["EUR_NOK"].resample("ME").mean(),   # gjennomsnittlig kurs
    "Q":       final_with_rates["Q"].resample("ME").last(),         # reell valutakurs (nivå)
    "d_pi":    final_with_rates["d_pi"].resample("ME").last(),      # inflasjonsdifferanse
    "dI_t":    final_with_rates["dI_t"].resample("ME").last(),      # rentedifferanse
})

# Fjern NaN som kan oppstå i starten
final_monthly = final_monthly.dropna()

# --- Sjekk ---
print(final_monthly.head(12))
print(final_monthly.tail(12))
print(f"\nTotalt rader (månedlig): {len(final_monthly)} | "
      f"Periode: {final_monthly.index.min().date()} → {final_monthly.index.max().date()}")

# (Valgfritt) lagre
# final_monthly.to_csv("final_dataset_monthly.csv", index=True)


             EUR_NOK         Q      d_pi  dI_t
DATE                                          
2000-01-31  8.082500  2.030652  0.001817  5.00
2000-02-29  8.099034  2.028587  0.001817  5.00
2000-03-31  8.112532  2.030130 -0.000553  5.00
2000-04-30  8.149967  2.032116  0.005282  5.00
2000-05-31  8.194226  2.052451 -0.001189  5.00
2000-06-30  8.255183  2.037880  0.000017  5.00
2000-07-31  8.175952  2.044609 -0.005020  5.00
2000-08-31  8.096790  2.030095 -0.000788  5.00
2000-09-30  8.030400  2.020539  0.003469  5.00
2000-10-31  8.003935  2.001680 -0.000261  3.25
2000-11-30  7.996533  2.023794  0.000365  3.25
2000-12-31  8.143081  2.050894 -0.004871  3.25
              EUR_NOK         Q      d_pi  dI_t
DATE                                           
2024-11-30  11.741250  2.420865  0.003233  0.75
2024-12-31  11.755200  2.431128 -0.000508  1.00
2025-01-31  11.746923  2.430318 -0.004094  1.00
2025-02-28  11.658339  2.423597  0.005630  1.25
2025-03-31  11.551219  2.399005 -0.002335  1.50
2025-0

#Endogene

##Vix

In [11]:
import pandas as pd
import numpy as np

# --- 1) Hent VIX (FRED CSV) ---
vix_url = ("https://fred.stlouisfed.org/graph/fredgraph.csv?"
           "bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450"
           "&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0"
           "&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=VIXCLS&scale=left&cosd=1990-01-02"
           "&coed=2025-09-25&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3"
           "&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily%2C%20Close&fam=avg&fgst=lin&fgsnd=2020-02-01"
           "&line_index=1&transformation=lin&vintage_date=2025-09-26&revision_date=2025-09-26&nd=1990-01-02")

vix_raw = pd.read_csv(vix_url)

# --- 2) Normaliser kolonnenavn ---
vix_raw.columns = [c.strip() for c in vix_raw.columns]
date_col = "observation_date" if "observation_date" in vix_raw.columns else vix_raw.columns[0]
val_col  = "VIXCLS" if "VIXCLS" in vix_raw.columns else vix_raw.columns[1]

# --- 3) Parser dato robust: prøv ISO først, fall tilbake til dag-først ---
def parse_dates_robust(s):
    s = pd.to_datetime(s, errors="coerce", utc=False)
    if s.isna().mean() > 0.5:
        # Fallback for formater som 02/01/1990 (dd/mm/yyyy)
        s = pd.to_datetime(vix_raw[date_col], errors="coerce", utc=False, dayfirst=True)
    return s

vix_raw["DATE"] = parse_dates_robust(vix_raw[date_col])

# --- 4) Rens verdier (FRED bruker ofte '.' for missing) ---
vix_raw[val_col] = vix_raw[val_col].replace(".", np.nan)
vix_raw["VIX"]   = pd.to_numeric(vix_raw[val_col], errors="coerce")

vix = (vix_raw[["DATE", "VIX"]]
       .dropna(subset=["DATE"])
       .sort_values("DATE")
       .set_index("DATE"))

# --- 5) Reindekser til daglig kalender og ffill/bfill ---
if "merged" in globals() and isinstance(merged.index, pd.DatetimeIndex):
    target_start = merged.index.min()
    target_end   = merged.index.max()
else:
    target_start = vix.index.min()
    target_end   = vix.index.max()

target_idx = pd.date_range(start=target_start, end=target_end, freq="D")

vix_daily = (vix.reindex(target_idx)
                .ffill()   # fyll handelsfrie dager med siste kjente
                .bfill())  # fyll start hvis nødvendig
vix_daily.index.name = "DATE"

# --- 6) (Valgfritt) merge inn i datasettet ditt ---
# merged = merged.merge(vix_daily, left_index=True, right_index=True, how="left")

# --- 7) Sjekk ---
print(vix_daily.head(10))
print(vix_daily.tail(10))
print(f"\nVIX daglig: {vix_daily.index.min().date()} → {vix_daily.index.max().date()} | NaN: {vix_daily['VIX'].isna().sum()}")


              VIX
DATE             
2000-01-03  24.21
2000-01-04  27.01
2000-01-05  26.41
2000-01-06  25.73
2000-01-07  21.72
2000-01-08  21.72
2000-01-09  21.72
2000-01-10  21.71
2000-01-11  22.50
2000-01-12  22.84
              VIX
DATE             
2025-10-18  16.74
2025-10-19  16.74
2025-10-20  16.74
2025-10-21  16.74
2025-10-22  16.74
2025-10-23  16.74
2025-10-24  16.74
2025-10-25  16.74
2025-10-26  16.74
2025-10-27  16.74

VIX daglig: 2000-01-03 → 2025-10-27 | NaN: 0


##Brent Oil


In [12]:

!pip -q install pandas requests

# %%
import io
from datetime import datetime, timezone
import pandas as pd
import requests

APP_TOKEN = "laCqAPM9Wo1SggEqlGFBAdssN"  # X-App-Token
CSV_ENDPOINT = "https://agtransport.usda.gov/api/v3/views/b3w8-gxpm/query.csv"

date_from = "1999-01-01T00:00:00.000"
date_to   = datetime.now(timezone.utc).strftime("%Y-%m-%dT23:59:59.999")

params = {
    "select": "date, brent",  # <-- kun disse feltene
    "where": f"date between '{date_from}' and '{date_to}'",
    "order": "date ASC",
}
headers = {"X-App-Token": APP_TOKEN}

resp = requests.get(CSV_ENDPOINT, headers=headers, params=params, timeout=60)
resp.raise_for_status()

df = pd.read_csv(io.BytesIO(resp.content))

# Sikre riktige typer
df["date"]  = pd.to_datetime(df["date"], errors="coerce", utc=True).dt.tz_convert(None)
df["brent"] = pd.to_numeric(df["brent"], errors="coerce")

# Kun de to kolonnene (skulle allerede være det, men for sikkerhets skyld):
df = df[["date", "brent"]]

print(f"Rader hentet: {len(df):,}")
display(df.head(5))
display(df.tail(5))

# Valgfritt: lagre
# df.to_csv("brent_date_brent_1999_to_today.csv", index=False)


Rader hentet: 9,729


,date,brent
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63


,date,brent
9724,2025-09-16,69.69
9725,2025-09-17,69.19
9726,2025-09-18,67.83
9727,2025-09-19,67.05
9728,2025-09-22,66.87


In [13]:

import pandas as pd

# Forutsetter at df inneholder kolonnene 'date' (datetime) og 'brent' (float)
df_ff = df.copy().sort_values("date")
df_ff = df_ff.set_index("date")

# Lag komplett daglig indeks fra min til maks dato
full_idx = pd.date_range(df_ff.index.min(), df_ff.index.max(), freq="D")

# Reindekser til daglig frekvens (setter NaN der pris mangler)
df_daily = df_ff.reindex(full_idx)

# Tell manglende før fylling
missing_before = df_daily["brent"].isna().sum()

# Forward-fill (fyller ikke før første observasjon; ev. ledende NaN beholdes)
df_daily["brent"] = df_daily["brent"].ffill()

# Valgfritt: fjern ledende NaN-rader dersom dataserien starter med hull
df_daily = df_daily[df_daily["brent"].notna()]

missing_after = df_daily["brent"].isna().sum()
filled_days = missing_before - missing_after

# Tilbake til vanlig DataFrame med 'date' som kolonne
df_daily = df_daily.rename_axis("date").reset_index()

print(f"Dager uten pris før ffill: {missing_before:,}")
print(f"Dager uten pris etter ffill: {missing_after:,}")
print(f"Dager fylt med ffill: {filled_days:,}")

display(df_daily.head(5))
display(df_daily.tail(5))


Dager uten pris før ffill: 4,277
Dager uten pris etter ffill: 0
Dager fylt med ffill: 4,277


,date,brent
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-23,18.55
4,1987-05-24,18.55


,date,brent
14001,2025-09-18,67.83
14002,2025-09-19,67.05
14003,2025-09-20,67.05
14004,2025-09-21,67.05
14005,2025-09-22,66.87


#Stoxx


In [16]:
import pandas as pd

URL = "https://raw.githubusercontent.com/bredeespelid/Data_MasterOppgave/refs/heads/main/StoxxEuro600.csv"

# Les rått som tekst for å sikre full kontroll på rensing
raw = pd.read_csv(URL, sep=",", dtype=str, encoding="utf-8")

# Trim kolonnenavn
raw.columns = raw.columns.str.strip()

# Parse dato: "dd.mm.yyyy kl. HH.MM.SS" -> datetime
dt = (
    raw["Date"]
    .astype(str)
    .str.replace(" kl. ", " ", regex=False)
)
date = pd.to_datetime(dt, format="%d.%m.%Y %H.%M.%S", errors="coerce")

# Rens Close -> StoxEurope: fjern NBSP/space, bytt komma->punktum, cast til float
vals = (
    raw["Close"]
    .astype(str)
    .str.replace("\u00A0", "", regex=False)   # NBSP
    .str.replace(" ", "", regex=False)        # vanlige mellomrom
    .str.replace(",", ".", regex=False)       # norsk desimal -> engelsk
    .replace({"": None})
)
stox = pd.to_numeric(vals, errors="coerce")

# Sett sammen til endelig df
df = pd.DataFrame({"Date": date, "StoxEurope": stox}).sort_values("Date").dropna(subset=["Date"]).set_index("Date")

# Rask kontroll
print("Antall NaN i StoxEurope:", df["StoxEurope"].isna().sum())
print("Første dato:", df.index.min().date(), "Siste dato:", df.index.max().date())
print(df.head(5))
print(df.tail(5))


Antall NaN i StoxEurope: 0
Første dato: 1998-07-17 Siste dato: 2025-09-26
                     StoxEurope
Date                           
1998-07-17 18:00:00      313.83
1998-07-20 18:00:00      315.00
1998-07-21 18:00:00      313.52
1998-07-22 18:00:00      308.13
1998-07-23 18:00:00      307.42
                     StoxEurope
Date                           
2025-09-22 18:00:00      553.40
2025-09-23 18:00:00      554.95
2025-09-24 18:00:00      553.88
2025-09-25 18:00:00      550.22
2025-09-26 18:00:00      554.52


In [17]:
# Reindekser til ukedager (uten helligdager) og forward-fill mangler
import pandas as pd

# 1) Lag ukedagsindeks på dato-nivå og legg på klokkeslett 18:00
start = df.index.min().normalize()
end = df.index.max().normalize()
bidx = pd.bdate_range(start=start, end=end, freq="B") + pd.Timedelta(hours=18)

# 2) Reindekser og ffill
df_ffill = df.reindex(bidx).ffill()
df_ffill.index.name = "Date"

# 3) Kontroll
print("Antall NaN etter ffill:", df_ffill["StoxEurope"].isna().sum())
print("Første dato:", df_ffill.index.min().date(), "Siste dato:", df_ffill.index.max().date())
print(df_ffill.head(5))
print(df_ffill.tail(5))

# Valgfritt: lagre
# df_ffill.to_csv("StoxEurope600_ffilled.csv", index_label="Date")


Antall NaN etter ffill: 0
Første dato: 1998-07-17 Siste dato: 2025-09-26
                     StoxEurope
Date                           
1998-07-17 18:00:00      313.83
1998-07-20 18:00:00      315.00
1998-07-21 18:00:00      313.52
1998-07-22 18:00:00      308.13
1998-07-23 18:00:00      307.42
                     StoxEurope
Date                           
2025-09-22 18:00:00      553.40
2025-09-23 18:00:00      554.95
2025-09-24 18:00:00      553.88
2025-09-25 18:00:00      550.22
2025-09-26 18:00:00      554.52


#S&P500


In [18]:
import re
import requests
import pandas as pd

URL_SP = "https://raw.githubusercontent.com/bredeespelid/Data_MasterOppgave/refs/heads/main/S%26P.csv"

# --- Hjelpefunksjon for tallrensing ---
def _clean_number(s: str) -> str:
    s = (s.replace("\u00A0", "")   # NBSP
           .replace("\u202F", "")  # smal NBSP
           .replace(" ", ""))      # vanlige mellomrom
    if "." in s and "," in s:      # punktum=1000-sep, komma=desimal
        s = s.replace(".", "").replace(",", ".")
    else:
        s = s.replace(",", ".")
    return s

# --- 1) Hent råtekst og parse med regex ---
txt = requests.get(URL_SP, timeout=30).text
pat = re.compile(r'(\d{2}\.\d{2}\.\d{4})\s+kl\.\s+(\d{2}\.\d{2}\.\d{2}),\s*"?([^"\s]+)"?')
rows = [(f"{d} {t}", _clean_number(v)) for d, t, v in pat.findall(txt)]

# --- 2) Bygg DataFrame ---
df_sp = (
    pd.DataFrame(rows, columns=["Date", "SP500"])
      .assign(Date=lambda x: pd.to_datetime(x["Date"], format="%d.%m.%Y %H.%M.%S", errors="coerce").dt.normalize(),
              SP500=lambda x: pd.to_numeric(x["SP500"], errors="coerce"))
      .dropna(subset=["Date"])
      .drop_duplicates(subset=["Date"])
      .sort_values("Date")
      .set_index("Date")
)

# --- 3) Reindekser til alle ukedager (uten klokkeslett) og ffill ---
bidx_sp = pd.bdate_range(start=df_sp.index.min(), end=df_sp.index.max(), freq="B")
df_sp_ffill = df_sp.reindex(bidx_sp).ffill()
df_sp_ffill.index.name = "Date"

# --- 4) Kontroll ---
print("Antall NaN etter ffill:", df_sp_ffill["SP500"].isna().sum())
print("Første dato:", df_sp_ffill.index.min().date(), "Siste dato:", df_sp_ffill.index.max().date())
print(df_sp_ffill.head(5))
print(df_sp_ffill.tail(5))


Antall NaN etter ffill: 0
Første dato: 1996-11-18 Siste dato: 2025-09-26
             SP500
Date              
1996-11-18  737.02
1996-11-19  742.16
1996-11-20  742.16
1996-11-21  742.72
1996-11-22  748.70
              SP500
Date               
2025-09-22  6693.75
2025-09-23  6656.92
2025-09-24  6637.97
2025-09-25  6604.72
2025-09-26  6643.70


#DailyCombined

In [19]:
import pandas as pd

# ========= STOXX Europe 600: reindekser til ukedager og ffill =========
# Forutsetter at du allerede har 'df' med indeks=DateTime og kolonne 'StoxEurope' (uten NaN).
start = df.index.min().normalize()
end   = df.index.max().normalize()
bidx  = pd.bdate_range(start=start, end=end, freq="B") + pd.Timedelta(hours=18)

df_ffill = df.reindex(bidx).ffill()
df_ffill.index.name = "Date"

# Dato-normalisert for «date-only»-align
stox_idx = df_ffill.copy()
stox_idx.index = stox_idx.index.normalize()

# ========= SP500: bruk eksisterende df_sp_ffill (datoindeks uten klokkeslett) =========
# df_sp_ffill må finnes fra din tidligere blokk.
sp_idx = df_sp_ffill.copy()             # <<< SP500
sp_idx.index = sp_idx.index.normalize() # <<< SP500

# ========= Brent og VIX: klargjør som før =========
# 1) Brent (df_daily: ['date','brent'])
brent_idx = (
    df_daily.copy()
    .assign(date=pd.to_datetime(df_daily["date"], errors="coerce").dt.normalize())
    .dropna(subset=["date"])
    .set_index("date")
    .sort_index()[["brent"]]
)

# 2) VIX (vix_daily: index=dato, én kolonne som døpes 'VIX')
vix_idx = vix_daily.copy()
vix_idx.index = pd.to_datetime(vix_idx.index, errors="coerce").normalize()
vix_idx.index.name = "date"
vix_idx.columns = [str(c).strip().upper() for c in vix_idx.columns]
if "VIX" not in vix_idx.columns and len(vix_idx.columns) == 1:
    vix_idx = vix_idx.rename(columns={vix_idx.columns[0]: "VIX"})
vix_idx = vix_idx.sort_index()[["VIX"]]

# ========= Base: sørg for DatetimeIndex =========
base = final_with_rates.copy()
if not isinstance(base.index, pd.DatetimeIndex):
    base.index = pd.to_datetime(base.index, errors="coerce")
base = base.sort_index()

# Vi aligner på "dato" uavhengig av klokkeslett:
base_dates = base.index.normalize()

# ========= Align alle serier til base-datoer (ffill) =========
brent_aligned_bydate = brent_idx.reindex(base_dates, method="ffill")
vix_aligned_bydate   = vix_idx.reindex(base_dates,   method="ffill")
stox_aligned_bydate  = stox_idx.reindex(base_dates,  method="ffill")
sp_aligned_bydate    = sp_idx.reindex(base_dates,    method="ffill")  # <<< SP500

# Legg tilbake samme indeks som 'base' (med ev. klokkeslett)
brent_aligned = brent_aligned_bydate.set_index(base.index)
vix_aligned   = vix_aligned_bydate.set_index(base.index)
stox_aligned  = stox_aligned_bydate.set_index(base.index)
sp_aligned    = sp_aligned_bydate.set_index(base.index)               # <<< SP500

# ========= Rydd overlapp og join =========
for col in ["brent", "VIX", "vix", "StoxEurope", "SP500"]:            # <<< SP500
    if col in base.columns:
        base = base.drop(columns=[col])

final_with_rates = (
    base
    .join(brent_aligned, how="left")
    .join(vix_aligned,   how="left")
    .join(stox_aligned,  how="left")
    .join(sp_aligned,    how="left")                                   # <<< SP500
)

# ========= Kolonnerekkefølge (tilpass etter behov) =========
pref = [c for c in ["EUR_NOK", "Q", "d_pi", "dI_t"] if c in final_with_rates.columns]
cols = pref + [c for c in ["brent", "VIX", "StoxEurope", "SP500"] if c in final_with_rates.columns]  # <<< SP500
final_with_rates = final_with_rates[cols]

# ========= Sjekk =========
print(final_with_rates.head(12))
print(final_with_rates.tail(12))
print(
    f"\nTotalt rader: {len(final_with_rates):,} | "
    f"Periode: {final_with_rates.index.min().date()} → {final_with_rates.index.max().date()} | "
    f"NaN i brent: {final_with_rates['brent'].isna().sum():,} | "
    f"NaN i VIX: {final_with_rates['VIX'].isna().sum():,} | "
    f"NaN i StoxEurope: {final_with_rates['StoxEurope'].isna().sum():,} | "  # <<< SP500
    f"NaN i SP500: {final_with_rates['SP500'].isna().sum():,}"               # <<< SP500
)

final_with_rates.to_csv("final_with_rates.csv", index_label="Date")



            EUR_NOK         Q      d_pi  dI_t  brent    VIX  StoxEurope  \
DATE                                                                      
2000-01-31   8.0825  2.030652  0.001817   5.0  27.08  24.95      360.93   
2000-02-01   8.0730  2.027659  0.001817   5.0  27.35  23.45      366.71   
2000-02-02   8.0175  2.020760  0.001817   5.0  27.15  23.12      371.34   
2000-02-03   8.0475  2.024495  0.001817   5.0  27.60  22.01      376.29   
2000-02-04   8.0830  2.028897  0.001817   5.0  27.48  21.54      377.37   
2000-02-05   8.0830  2.028897  0.001817   5.0  27.48  21.54      377.37   
2000-02-06   8.0830  2.028897  0.001817   5.0  27.48  21.54      377.37   
2000-02-07   8.0590  2.025923  0.001817   5.0  27.94  22.79      374.20   
2000-02-08   8.0720  2.027535  0.001817   5.0  27.61  21.25      382.38   
2000-02-09   8.0825  2.028835  0.001817   5.0  27.44  22.90      382.35   
2000-02-10   8.0695  2.027225  0.001817   5.0  27.32  23.07      381.61   
2000-02-11   8.0395  2.02